In [1]:
import pandas as pd
from gemini import tag_topics
from models import variable_to_question


In [2]:
data = pd.read_parquet('data/encuesta.parquet').drop(columns=["infrastructure_changes_raw", "most_important_technology_raw", "improved_aspect_raw", "challenges_raw", "recomendations_raw"])

print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, 1 to 109
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   response_id                           109 non-null    string        
 1   start_response                        109 non-null    datetime64[ns]
 2   complete_response                     109 non-null    datetime64[ns]
 3   age                                   109 non-null    category      
 4   sex                                   109 non-null    category      
 5   years_studying                        109 non-null    int64         
 6   degree                                109 non-null    category      
 7   infrastructure_score                  109 non-null    category      
 8   notice_infrastructure_improvement     109 non-null    category      
 9   frecuency_digital_systems             109 non-null    category      
 10  technol

In [3]:
classification_input = data.loc[data['infrastructure_changes'].apply(lambda x : len(x) > 0),['response_id', "infrastructure_changes"]]
classification_input["infrastructure_changes"] = classification_input["infrastructure_changes"].apply(lambda x: x.tolist())
result = tag_topics(variable_to_question["infrastructure_changes"], classification_input.to_dict())

In [8]:
df = pd.DataFrame(result)

df["tag"].astype('category').cat.categories

Index(['Ampliación y Modernización Hospitalaria', 'Atención Especializada',
       'Atención y Comodidad del Paciente', 'Bioseguridad y Equipamiento',
       'Construcción de Nuevos Hospitales', 'Digitalización y Tecnología',
       'Equipamiento Médico', 'Mejoras en Instalaciones Físicas'],
      dtype='object')